#### Set up Dependencies and import any keys needed

In [1]:

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import os
import gmaps 
import us

from census import Census
# Import API key
from api_keys import g_key
from api_keys import weather_api_key
from api_keys import gas_api_key
from api_keys import eia_key
from api_keys import census_api_key
from api_keys import nps_key

#from config import census_api
#c = Census(census_api)
c = Census(census_api_key)
print (eia_key)

95abebd4884337a93dda412e59cda738


##### set up West States 

In [2]:
park = 'acad'
url = f'https://developer.nps.gov/api/v1/parks?parkCode={park}&api_key=parks_key'
print(url)

https://developer.nps.gov/api/v1/parks?parkCode=acad&api_key=parks_key


In [3]:

West = ['Colorado', 'Wyoming', 'Montana', 'Idaho', 'Washington', 'Oregon', 'Utah', 'Nevada', 'California', 'Alaska', 'Hawaii']



##### Make a call to census data to pull info for all states, put into df, rename columns
##### May not need but put it in a csv just in case

In [4]:

census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"), {'for': 'state:*'})



#Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "state": "State"})


#census_pd.head()
census_pd.to_csv("./output_data/census_data_allstates.csv", encoding="utf-8", index=False)
census_pd.head()

,Name,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Unemployment Count,State
0,Puerto Rico,19775.0,3468963.0,40.1,12081.0,1543220.0,220597.0,72
1,Alabama,46472.0,4850771.0,38.7,25746.0,849699.0,165391.0,01
2,Alaska,76114.0,738565.0,33.9,35065.0,73380.0,29548.0,02
3,Arizona,53510.0,6809946.0,37.2,27964.0,1128046.0,225911.0,04
4,Arkansas,43813.0,2977944.0,37.9,24426.0,524211.0,85128.0,05


##### This file was provided to us in a separate activity
##### Gives centroids of each state

In [5]:
# Read in the csv containing state centroid coordinates
centroids = pd.read_csv("./Resources/state_centroids.csv")
centroids.head()

,State,Latitude,Longitude
0,Alabama,32.7794,-86.8287
1,Alaska,64.0685,-152.2782
2,Arizona,34.2744,-111.6602
3,Arkansas,34.8938,-92.4426
4,California,37.1841,-119.4696


In [6]:
# create a dictionary of states and abbreviations
us_state_abbrev = {
'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}



##### Merge the centroids and census data to get a lat/lon for each state

In [7]:
# Merge the datasets using the sate columns
census_data = pd.merge(census_pd, centroids, how="left", left_on="Name", right_on="State")

# Save the updated dataframe as a csv
census_data.to_csv("./Resources/state_census_data.csv", encoding="utf-8", index=False)

#df['State'] = df['State'].map(us_state_abbrev).fillna(df['State'])

census_data['State_y'] = census_data['State_y'].map(us_state_abbrev).fillna(census_data['State_y'])
census_data.rename(columns={'State_y':'State Code'})
census_data.head()

,Name,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Unemployment Count,State_x,State_y,Latitude,Longitude
0,Puerto Rico,19775.0,3468963.0,40.1,12081.0,1543220.0,220597.0,72,Puerto Rico,18.2001,-66.6645
1,Alabama,46472.0,4850771.0,38.7,25746.0,849699.0,165391.0,01,AL,32.7794,-86.8287
2,Alaska,76114.0,738565.0,33.9,35065.0,73380.0,29548.0,02,AK,64.0685,-152.2782
3,Arizona,53510.0,6809946.0,37.2,27964.0,1128046.0,225911.0,04,AZ,34.2744,-111.6602
4,Arkansas,43813.0,2977944.0,37.9,24426.0,524211.0,85128.0,05,AR,34.8938,-92.4426


##### Grab population data and map it

In [8]:
# Convert pop to a list
population = census_data["Population"].tolist()

In [9]:
# Create a map using state centroid coordinates to set markers
marker_locations = census_data[['Latitude', 'Longitude']]

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Population: {pop}" for pop in population])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

##### Pull out State Population Lat Lon

In [10]:
# Create a datframe with just the data we  want for the population quesion
pop_df = census_data[['Name', 'Population', 'Latitude', 'Longitude']].copy()

pop_df['State Code'] = pop_df['Name'].map(us_state_abbrev).fillna(pop_df['Name'])
pop_df

,Name,Population,Latitude,Longitude,State Code
0,Puerto Rico,3468963.0,18.2001,-66.6645,Puerto Rico
1,Alabama,4850771.0,32.7794,-86.8287,AL
2,Alaska,738565.0,64.0685,-152.2782,AK
3,Arizona,6809946.0,34.2744,-111.6602,AZ
4,Arkansas,2977944.0,34.8938,-92.4426,AR
5,California,38982847.0,37.1841,-119.4696,CA
6,Colorado,5436519.0,38.9972,-105.5478,CO
7,Connecticut,3594478.0,41.6219,-72.7273,CT
8,District of Columbia,672391.0,38.9101,-77.0147,District of Columbia
9,Delaware,943732.0,38.9896,-75.5050,DE


### Parks Data

##### Example request for one park

In [11]:
# # Find the parks in each state
# park = "DENA"
# parks_key =""  #NOTE NEED A KEY
# url = f"https://developer.nps.gov/api/v1/parks?parkCode={park}&api_key=parks_key"

# #https://irmaservices.nps.gov/v2/rest/unit/

# data = requests.get(url).json()

# # Prepare and execute output
# print(data["data"][0]["fullName"] + " can be found at " + data["data"][0]["latLong"] + ".")


In [12]:
URL = 'https://irmaservices.nps.gov/v2/rest/unit/?unitCodes={*}&format=json'
response = requests.get(URL).json()

#print(response)

len(response)
# months = ['January', 'February', 'March', 'April', 'May', 'June', 'July' 'August', 'September', 'October', 'November', 'December'] 
# for month in range(0,len(response)-1):
#     print(f"{unitcode} National Park had {response[month]['RecreationVisitors']} visitors" +
#           f" in the month of {months[month]} in 2019")


# set up dataframe column names
column_names = ['FullName',
                'UnitCode',
                'UnitName',
                'StateCode']
    
#print(response)
national_parks = []
for park in response:
    if(park['UnitDesignationName'] == 'National Park'):
#         print(f" {park['UnitDesignationName']}:{park['UnitCode']}: {park['FullName']} -" +
#               f" region {park['Region']},{park['StateCodes']} ")
        national_parks.append([park['FullName'], 
                          park['UnitCode'], 
                          park['UnitName'],
                          park['StateCodes']])
#park.keys()
#dict_keys(['UnitCode', 'UnitName', 'FullName', 
# 'UnitDesignationCode', 'UnitDesignationName', 'UnitSubTypeCode', 
#'UnitSubTypeName', 'UnitLifecycle', 'Network', 'NetworkName', 'Region', 'RegionName', 'StateCodes'])
national_parks
np_df = pd.DataFrame(national_parks, columns=column_names)
np_df

,FullName,UnitCode,UnitName,StateCode
0,Acadia National Park,ACAD,Acadia,[ME]
1,Arches National Park,ARCH,Arches,[UT]
2,Badlands National Park,BADL,Badlands,[SD]
3,Big Bend National Park,BIBE,Big Bend,[TX]
4,Biscayne National Park,BISC,Biscayne,[FL]
...,...,...,...,...
58,Wind Cave National Park,WICA,Wind Cave,[SD]
59,Wrangell-St. Elias National Park,WRSG,Wrangell-St. Elias,[AK]
60,Yellowstone National Park,YELL,Yellowstone,"[ID, MT, WY]"
61,Yosemite National Park,YOSE,Yosemite,[CA]


### Make a row for eash state in the list 
* census_data = pd.merge(census_pd, centroids, how="left", left_on="Name", right_on="State")


In [13]:
lst_col = 'StateCode'

r = pd.DataFrame({
      col:np.repeat(np_df[col].values, np_df[lst_col].str.len())
      for col in np_df.columns.drop(lst_col)}
    ).assign(**{lst_col:np.concatenate(np_df[lst_col].values)})[np_df.columns]
r

,FullName,UnitCode,UnitName,StateCode
0,Acadia National Park,ACAD,Acadia,ME
1,Arches National Park,ARCH,Arches,UT
2,Badlands National Park,BADL,Badlands,SD
3,Big Bend National Park,BIBE,Big Bend,TX
4,Biscayne National Park,BISC,Biscayne,FL
...,...,...,...,...
63,Yellowstone National Park,YELL,Yellowstone,ID
64,Yellowstone National Park,YELL,Yellowstone,MT
65,Yellowstone National Park,YELL,Yellowstone,WY
66,Yosemite National Park,YOSE,Yosemite,CA


##### Map state abbreviation state name and merge - still figuring this one out

In [14]:
pop_df.head(3)
        
pop_by_park = pd.merge(pop_df,r, how="outer", left_on="State Code", right_on="StateCode")
pop_by_park


,Name,Population,Latitude,Longitude,State Code,FullName,UnitCode,UnitName,StateCode
0,Puerto Rico,3468963.0,18.2001,-66.6645,Puerto Rico,NaN,NaN,NaN,NaN
1,Alabama,4850771.0,32.7794,-86.8287,AL,NaN,NaN,NaN,NaN
2,Alaska,738565.0,64.0685,-152.2782,AK,Denali National Park,DENG,Denali,AK
3,Alaska,738565.0,64.0685,-152.2782,AK,Gates of the Arctic National Park,GAAG,Gates of the Arctic,AK
4,Alaska,738565.0,64.0685,-152.2782,AK,Glacier Bay National Park,GLBG,Glacier Bay,AK
...,...,...,...,...,...,...,...,...,...
85,Wyoming,583200.0,42.9957,-107.5512,WY,Grand Teton National Park,GRTE,Grand Teton,WY
86,Wyoming,583200.0,42.9957,-107.5512,WY,Yellowstone National Park,YELL,Yellowstone,WY
87,Wisconsin,5763217.0,44.6243,-89.9941,WI,NaN,NaN,NaN,NaN
88,NaN,NaN,NaN,NaN,NaN,National Park of American Samoa,NPSA,National Park of American Samoa,AS


In [15]:
# Save the updated dataframe as a csv
np_df.to_csv("./Resources/state_park_data.csv", encoding="utf-8", index=False)
#np_df.head()

In [16]:
#notes some are empty
unitcode = "COSW" #Rocky Mountain National Park (unitcode)
URL = f'https://irmaservices.nps.gov/v3/rest/stats/visitation?unitCodes={unitcode}&startMonth=01&startYear=2019&endMonth=12&endYear=2019&format=json'
print(URL)

https://irmaservices.nps.gov/v3/rest/stats/visitation?unitCodes=COSW&startMonth=01&startYear=2019&endMonth=12&endYear=2019&format=json


#### PRINT the LAT/LON of each park

In [42]:
def convert_lat_lon(latlon):
    latlon = (data["data"][0]["latLong"])

    lat,lon = latlon.split(", ")

    l, lat=lat.split(':')
    l, lon=lon.split(':')

    lat = float(lat)
    lon = float(lon)
    return([lat, lon])
 

In [43]:
### Testing convert_lat_lon function
# ll = convert_lat_lon('lat:37.29839254, long:-113.0265138')
# print(ll[0])
# print(ll[1])

37.29839254
-113.0265138


In [50]:
from api_keys import nps_key
parks_key = nps_key #NOTE NEED A KEY
latlon=[]
for park in np_df['UnitCode']:
    url = f"https://developer.nps.gov/api/v1/parks?parkCode={park}&api_key={nps_key}"
    try:
        data = requests.get(url).json()
        # Prepare and execute output
        # print(data["data"][0]["fullName"] + " can be found at " + data["data"][0]["latLong"] + ".")
        ll = convert_lat_lon(data["data"][0]["latLong"])
        latlon.append([data["data"][0]["fullName"], ll[0], ll[1]])
    except:
        print(f"Empty Cell...Skipping {park}")

park_lat_lon_df = pd.DataFrame(latlon, columns = ['Park', 'Lat', 'Lon'])
park_lat_lon_df

Empty Cell...Skipping COSW
Empty Cell...Skipping DENG
Empty Cell...Skipping GAAG
Empty Cell...Skipping GLBG
Empty Cell...Skipping GRDG
Empty Cell...Skipping KATG
Empty Cell...Skipping KICA
Empty Cell...Skipping LACG
Empty Cell...Skipping SEQU
Empty Cell...Skipping WRSG


,Park,Lat,Lon
0,Acadia National Park,44.307775,-68.300633
1,Arches National Park,38.722618,-109.586367
2,Badlands National Park,43.685848,-102.482942
3,Big Bend National Park,29.298178,-103.229790
4,Biscayne National Park,25.490587,-80.210239
5,Black Canyon Of The Gunnison National Park,38.577799,-107.724276
6,Bryce Canyon National Park,37.583991,-112.182669
7,Canyonlands National Park,38.245558,-109.880162
8,Capitol Reef National Park,38.282165,-111.247048
9,Carlsbad Caverns National Park,32.140895,-104.552969


### MAP the PARKs

In [52]:
# Create a map using state centroid coordinates to set markers
marker_locations = park_lat_lon_df[['Lat', 'Lon']]
# Create a marker_layer using the park list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Park: {park}" for park in park_lat_lon_df['Park']])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
for unitcode in np_df['UnitCode']:
    print(f"Unit Code = {unitcode}")

In [ ]:
column_names = ['UnitCode','Month', 'Visitors']
query_year = "2019"
park_visitor_data = []

months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'] 
# for month in range(0, 12):
#         print(f"{months[month]}")
len(months)
for unitcode in np_df['UnitCode']:
    #print(f"Processing Unit Code = {unitcode} for {query_year}...")   
    # construct the query
    URL = f'https://irmaservices.nps.gov/v3/rest/stats/visitation?unitCodes={unitcode}&startMonth=01&startYear={query_year}&endMonth=12&endYear={query_year}&format=json'

    response = requests.get(URL).json()

    for month in range(0,len(response)):
     #     print(f"{month}")
    #     print(f"{unitcode} National Park had {response[month]['RecreationVisitors']} visitors" +
    #           f" in the month of {months[month]} in 2019")
          park_visitor_data.append([unitcode, 
                          month,
                          response[month]['RecreationVisitors']])
          
# # print("Processing complete")    
visitor_df = pd.DataFrame(park_visitor_data, columns=column_names)
visitor_df

In [ ]:
by_year = visitor_df.groupby('UnitCode').sum()
by_year

In [ ]:
# Save the updated dataframe as a csv
visitor_df.to_csv("./Resources/state_park_visitor_data.csv", encoding="utf-8", index=False)
visitor_df.head()  #VALIDATED AGAINST PARK WEBSITE DATA

In [ ]:
gas_url = "http://devapi.mygasfeed.com/"
apikey = "rfej9napna"  #Waiting for own API KEY THIS IS GENERIC

#houston 29.7604° N, 95.3698° W
Latitude   = "29.7604"
Longitude  = "-95.3698"
distance   = 30          # Valid values must be less than 50
fuel_type  =  "reg"  # Valid values  reg,mid,pre,diesel

query = f"stations/radius/{Latitude}/{Longitude}/{distance}/{fuel_type}/price/{apikey}.json?"

url = gas_url+query
print(url)   #uncomment to test url by clicking on it 

response = requests.get(url).json()

print(response)
response

In [ ]:
print(f"{eia_key}")

In [ ]:
#http://api.eia.gov/series/?api_key=YOUR_API_KEY_HERE&series_id=TOTAL.MGUCUUS.M
 # WAITING ON A KEY
url= f"http://api.eia.gov/category/?api_key={eia_key}&category_id=711295"
url = f"http://api.eia.gov/series/?api_key={eihttp://api.eia.gov/series/?api_key=95abebd4884337a93dda412e59cda738&series_id=TOTAL.MGUCUUS.Ma_key}&series_id=TOTAL.MGUCUUS.M"
print(url)
